In [2]:
import pandas as pd
df = pd.read_csv('dataset/prabowo_23-05.csv')
df1 = pd.read_csv('dataset/prabowo_23-06.csv')
df2 = pd.read_csv('dataset/prabowo_23-07.csv')
df3 = pd.read_csv('dataset/prabowo_23-08.csv')
df4 = pd.read_csv('prabowo_text.csv')

df = pd.concat([df, df1, df2, df3, df4], ignore_index=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12213 entries, 0 to 12212
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   conversation_id_str      8078 non-null   float64
 1   created_at               8078 non-null   object 
 2   favorite_count           8078 non-null   float64
 3   full_text                12213 non-null  object 
 4   id_str                   8078 non-null   float64
 5   image_url                1944 non-null   object 
 6   in_reply_to_screen_name  4076 non-null   object 
 7   lang                     8078 non-null   object 
 8   location                 0 non-null      float64
 9   quote_count              8078 non-null   float64
 10  reply_count              8078 non-null   float64
 11  retweet_count            8078 non-null   float64
 12  tweet_url                8078 non-null   object 
 13  user_id_str              8078 non-null   float64
 14  username              

In [6]:
df = df['full_text']

In [7]:
df.duplicated().sum()


1282

In [8]:
df.info()

<class 'pandas.core.series.Series'>
RangeIndex: 12213 entries, 0 to 12212
Series name: full_text
Non-Null Count  Dtype 
--------------  ----- 
12213 non-null  object
dtypes: object(1)
memory usage: 95.5+ KB


In [9]:
df = df.drop_duplicates()

In [11]:
df.to_csv('dataset/text/prabowo_text11000.csv')

In [4]:
df = pd.read_csv('dataset/text/prabowo_mdughol.csv')
df.drop(columns=['Unnamed: 0', 'label'], inplace=True)
df.head()

,full_text,text_processed,text_processed1
0,OK gas ...Indonesia Emas nih pak @prabowo @gib...,gas indonesia emas prabowo berkas kerja keras ...,ok gas indonesia emas nih pak prabowo gibran t...
1,@uglygrunge1798 @Heraloebss @prabowo @gibran_t...,prabowo ngga hubung pilih presiden wapres puas...,uglygrunge heraloebss prabowo gibran tweet ngg...
2,@DedynurPalakka Kinerja? Emang pernah dikasih ...,kerja emang kasih tugas misi prabowo,dedynurpalakka kinerja emang pernah dikasih tu...
3,@txtdrimedia Kinerja?? Emang dikasi tugas apa ...,kerja emang kasi tugas prabowo,txtdrimedia kinerja emang dikasi tugas apa sm ...
4,@uglygrunge1798 @Heraloebss @prabowo @gibran_t...,prabowo demokrasi tupoksi kerja ndasmu pimpin ...,uglygrunge heraloebss prabowo gibran tweet dem...


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import pandas as pd

# Load model IndoBERT klasifikasi sentimen (pre-trained)
model_name = "indobenchmark/indobert-base-p1"  # atau model fine-tuned
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained("w11wo/indonesian-roberta-base-sentiment-classifier")  # contoh fine-tuned sentiment model

# Buat pipeline klasifikasi
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Prediksi otomatis
predictions = df['text_processed1'].apply(lambda x: nlp(x)[0])  # ambil hasil top-1

# Tambahkan label ke dataframe
df['label'] = predictions.apply(lambda x: x['label'])
df['score'] = predictions.apply(lambda x: x['score'])

# Simpan hasil
df.head()


,full_text,text_processed,text_processed1,label,score
0,OK gas ...Indonesia Emas nih pak @prabowo @gib...,gas indonesia emas prabowo berkas kerja keras ...,ok gas indonesia emas nih pak prabowo gibran t...,neutral,0.963868
1,@uglygrunge1798 @Heraloebss @prabowo @gibran_t...,prabowo ngga hubung pilih presiden wapres puas...,uglygrunge heraloebss prabowo gibran tweet ngg...,neutral,0.426632
2,@DedynurPalakka Kinerja? Emang pernah dikasih ...,kerja emang kasih tugas misi prabowo,dedynurpalakka kinerja emang pernah dikasih tu...,neutral,0.994963
3,@txtdrimedia Kinerja?? Emang dikasi tugas apa ...,kerja emang kasi tugas prabowo,txtdrimedia kinerja emang dikasi tugas apa sm ...,neutral,0.588368
4,@uglygrunge1798 @Heraloebss @prabowo @gibran_t...,prabowo demokrasi tupoksi kerja ndasmu pimpin ...,uglygrunge heraloebss prabowo gibran tweet dem...,neutral,0.547282


In [6]:
df['label'].value_counts()

label
neutral     6197
negative    3168
positive    1566
Name: count, dtype: int64

In [7]:
df.describe()

,score
count,10931.000000
mean,0.827528
std,0.172673
min,0.337372
25%,0.702148
50%,0.895057
75%,0.977465
max,0.999278


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10931 entries, 0 to 10930
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  10931 non-null  int64  
 1   full_text   10931 non-null  object 
 2   label       10931 non-null  object 
 3   score       10931 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 341.7+ KB


In [8]:
df.to_csv('dataset/text/prabowo_w11wo.csv', index=False)

In [26]:
df_copy = df.copy()
df_copy = df_copy[df_copy['score'] >= 0.820]
df_copy['label'].value_counts()

label
neutral     4922
negative    1275
positive     426
Name: count, dtype: int64

In [35]:
df_copy1 = df.copy()
df_copy1 = df_copy1[(df_copy1['score'] >= 0.95) & (df_copy1['label'] == 'negative')]
df_copy1['label'].value_counts()

label
negative    650
Name: count, dtype: int64

In [44]:
df_copy2 = df.copy()
df_copy2 = df_copy2[(df_copy2['score'] >= 0.995) & (df_copy2['label'] == 'neutral')]
df_copy2['label'].value_counts()

label
neutral    557
Name: count, dtype: int64

In [47]:
df_copy = df.copy()
df_copy = df_copy[
    ((df_copy['label'] == 'negative') & (df_copy['score'] >= 0.95)) |
    ((df_copy['label'] == 'neutral') & (df_copy['score'] >= 0.995)) |
    ((df_copy['label'] == 'positive') & (df_copy['score'] >= 0.82))
]

df_copy['label'].value_counts()


label
negative    650
neutral     557
positive    426
Name: count, dtype: int64

In [48]:
df_copy.describe()

,Unnamed: 0,score
count,1633.000000,1633.000000
mean,6077.633803,0.967817
std,3436.886165,0.044255
min,2.000000,0.820064
25%,3190.000000,0.959729
50%,6403.000000,0.988456
75%,8621.000000,0.996301
max,12211.000000,0.998918


In [50]:
df_copy.to_csv('dataset/text/prabowo_text_score82.csv')